**This workbook will aim at implementing an example use this cryptobacktest library**

The strategy we aim to implement is an options-based approach designed to capture potential increases in the volatility of the underlying asset. Specifically, we will utilize a strategy known as a straddle, which involves simultaneously purchasing a call and a put option with the same strike price and expiration date. This setup allows us to profit from significant price movements in either direction. To identify potential opportunities, we will focus on periods of low volatility, anticipating that an increase in volatility may follow.

In [ ]:
from cryptobacktest.volatility import Volatility
from cryptobacktest.option import Option
from cryptobacktest.volatilitysignal import StrictVolatilitySignal

from pybacktestchain.data_module import DataModule
from pybacktestchain.data_module import get_stock_data
import matplotlib.pyplot as plt

#from pybacktestchain.broker import Broker
#from pybacktestchain.blockchain import Blockchain
import pandas as pd
from cryptobacktest.straddlebacktest import StraddleBacktest
from cryptobacktest.straddlebroker import StraddleBroker
from cryptobacktest.straddleinfo import StraddleInformation

In [ ]:
# Define the date range
start_date = "2016-01-01"
end_date = "2024-12-31"

# Import historical data thanks to a pybacktestchain function
btc_data = get_stock_data('BTC-USD', start_date, end_date)
btc_data

We will be working on the closing price so we will only keep them.

In [ ]:
btc_close = btc_data[['Date', 'Close']].copy()

# Ensure the date column is a datetime format
btc_close['Date'] = pd.to_datetime(btc_close['Date'])

btc_close

In [ ]:
# Plot the Price
plt.figure(figsize=(12, 6))
plt.plot(btc_close['Date'], btc_close['Close'], label='Daily closing Price of BTCUSD', color='blue')
plt.title('Daily closing Price of BTCUSD', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('USD', fontsize=12)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

Initialize the Volatility class to compute a rolling monthly volatility

In [ ]:
volatility_calculator = Volatility(window=360,factor=30)

# Compute monthly rolling volatility
btc_close = volatility_calculator.compute_volatility(btc_close, price_column='Close', date_column='Date')

In [ ]:
btc_close

In [ ]:
# Plot the rolling monthly volatility
plt.figure(figsize=(12, 6))
plt.plot(btc_close['Date'], btc_close['Volatility'], label='Volatility', color='blue')
plt.title('Rolling Monthly Volatility of BTC', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Monthly Volatility', fontsize=12)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

Compute the 30-day maturity straddle price for each dates


In [ ]:
btc_close['StraddlePrice'] = btc_close.apply(
    lambda row: Option.compute_straddle(
        underlying_price=row['Close'],
        implied_volatility=row['Volatility'],
        time_to_maturity=30,  # 30-day maturity
        interest_rate=0.0,    # No interest rate
        dividend_yield=0.0    # No dividend yield
    ) if not pd.isna(row['Volatility']) else None,
    axis=1
)

btc_close

We now aim at computing potential buy straddle signals. To do this we will initialize our StrictVolatilitySignal class using the following parameters: a short-term volatility period of 1 month and a long-term volatility period of 3 months. Also we will only be interested in it if it's at least 5% lower for 3 consecutive days. Finally, we will take a maximum of 1 trade each 3 months to avoid taking many positions in the case the volatility decreases a lot very rapidly.

In [ ]:
strict_signal_with_frequency = StrictVolatilitySignal(
    vol_column="Volatility", 
    date_column="Date", 
    short_window=30, 
    long_window=90, 
    threshold_factor=0.95, 
    min_days=3, 
    trading_frequency=90)

In [ ]:
# Generating the sinals on our df 
btc_close_with_frequency_signals = strict_signal_with_frequency.generate_signals(btc_close)

In [ ]:
# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(btc_close_with_frequency_signals['Date'], btc_close_with_frequency_signals['Volatility'], label='Volatility', color='blue')
plt.scatter(
    btc_close_with_frequency_signals['Date'][btc_close_with_frequency_signals['Signal'] == 1],
    btc_close_with_frequency_signals['Volatility'][btc_close_with_frequency_signals['Signal'] == 1],
    color='red', label='Low Volatility Signal'
)
plt.title('Volatility and trading signals')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.legend()
plt.grid()
plt.show()


Now by looking at volatility, spot and where our signals are, it's very hard to know if we would make money. 

Indeed straddles are expensive, so to know if this straddle strategy calibrated with our parameters would make money, we need to run a backtest ! 

Hence it's from here that will try to make the link between what I developed and the pybacktestchain created in class. As pybacktestchain was initially made for stocks and not options we will have to modify it a bit to fit our constraints. 

Indeed, with our option strategy we don't aim to rebalance our portfolio as we will keep positions until maturity and we will have no "risk model" as once bought we will hold them.

In [ ]:
btc_close_with_frequency_signals

Now our dataset and strategy are ready to be backtested. We will initialiaze all the necessary classes and variables.

In [ ]:
# Initialize the data module
data_module = DataModule(data=btc_close_with_frequency_signals)


In [ ]:
# Initialize the information class
info = StraddleInformation(
    data_module=data_module,
    s=timedelta(days=360),
    time_column="Date",
    adj_close_column="Close"
)

In [ ]:
# initialize the backtest
backtest = StraddleBacktest(
    initial_date=btc_close_with_frequency_signals["Date"].min(),
    final_date=btc_close_with_frequency_signals["Date"].max(),
    data_module=data_module,
    initial_cash=10000,  # Starting cash
    allocation_percent=0.1,  # Allocate 10% of portfolio to buying straddles at each date x, sort of risk parameter
)

Everything is now ready to run the backtest on the defined strategy.

In [ ]:
backtest.run_backtest(information_class=info)

We will grab the results: portfolio value & P&L in a dataframe in order to do some graphs based on the strategy

In [ ]:
portfolio_history, realized_pnl = backtest.get_results()

# Get all potential trade dates
trade_dates = btc_close_with_frequency_signals.loc[btc_close_with_frequency_signals["Signal"] == 1, "Date"]

#  only dates where trades occurred are considered (if cash < straddleprice, no trade)
successful_trade_dates = trade_dates.iloc[:len(realized_pnl)].reset_index(drop=True)

# Show pnl & cumulative pnl where trades were made

realized_pnl_df = pd.DataFrame({
    "Date": successful_trade_dates,
    "RealizedPnL": realized_pnl,
})
realized_pnl_df["CumulativePnL"] = realized_pnl_df["RealizedPnL"].cumsum()

realized_pnl_df



In [ ]:
# Create graphs and parameters
fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.plot(
    btc_close_with_frequency_signals["Date"],
    btc_close_with_frequency_signals["Close"],
    label="Price",
    color="blue",
    linewidth=1.5,
)
ax1.scatter(
    btc_close_with_frequency_signals.loc[btc_close_with_frequency_signals["Signal"] == 1, "Date"],
    btc_close_with_frequency_signals.loc[btc_close_with_frequency_signals["Signal"] == 1, "Close"],
    color="red",
    label="Trade Signals",
    zorder=5,
)
ax1.set_xlabel("Date", fontsize=12)
ax1.set_ylabel("Price", fontsize=12, color="blue")
ax1.tick_params(axis="y", labelcolor="blue")

ax2 = ax1.twinx()
ax2.scatter(
    realized_pnl_df["Date"],
    realized_pnl_df["RealizedPnL"],
    color="green",
    label="Realized PnL",
    zorder=6,
    s=50,
)
ax2.set_ylabel("Realized P&L", fontsize=12, color="green")
ax2.tick_params(axis="y", labelcolor="green")
fig.suptitle("Price, Trading Signals, and Realized P&L", fontsize=16)
ax1.legend(loc="upper left")
ax2.legend(loc="upper right")

ax1.grid()

plt.show()


Cumulative realized P&L:


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(realized_pnl_df["Date"], realized_pnl_df["CumulativePnL"], label="Cumulative Realized P&L", color="green", linewidth=2)
plt.title("Cumulative Realized P&L from our  strategy", fontsize=16)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Cumulative P&L", fontsize=12)
plt.legend()
plt.grid()
plt.show()

**Analysis of the P&L of this strategy:**

The results show that this strategy applied to BTC appears highly profitable. 

However, we need to be careful: 

Here, we used historical volatility as a proxy for implied volatility due to data limitations, which may not reflect real market conditions. 

Additionally, past performance of cryptocurrencies, driven by unique market conditions and rapid growth, may not be indicative of future performance.